In [1]:
import time
import os
import time
import json
import numpy as np

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

# DATA

In [2]:
# Cargar el dataset desde el archivo .npy
path2 = '../DatasetVGG19/train.npy'

data_train = np.load(path2)

# UTILS

In [3]:
def unpack(array):
    x = array[:,:-2]
    y = array[:,-2:]
    y = np.argmax(y, axis=1)
    return x,y

def model_score(X,y,model,dataset):
    accuracy = model.score(X, y)
    print(f"Precisión del modelo {dataset}: {accuracy * 100:.2f}%")
    return accuracy

def writeJson(tiempo,accu,metod):
    nuevo_registro = {
        'Tipo Ejecucion' : "CPU",
        'Metodo' : metod,
        'Epochs':20,
        'tiempo_ejecucion': tiempo,
        'accuracy' : accu
    }
    nombre_archivo = '../CPU_tiempos.json'

    # Leer el contenido existente, si el archivo ya existe
    if os.path.exists(nombre_archivo):
        with open(nombre_archivo, 'r') as archivo:
            try:
                registros = json.load(archivo)
            except json.JSONDecodeError:
                registros = []
    else:
        registros = []

    # Agregar el nuevo registro
    registros.append(nuevo_registro)

    # Guardar los registros actualizados en el archivo JSON
    with open(nombre_archivo, 'w') as archivo:
        json.dump(registros, archivo, indent=4)

# TRAINING DATA

In [4]:
X_train,y_train = unpack(data_train)

# MLPClassifier

[Documentacion](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)

In [5]:
def mlpc_init():
    return MLPClassifier(hidden_layer_sizes=(500, 200), activation='relu', max_iter=50, solver='sgd', verbose=True, early_stopping=True)

In [6]:
print("Entrenamiento MLP Classifier")

# Define hyperparameter grid
param_grid = {
    'hidden_layer_sizes': [(1000, 500, 2), (1000, 100, 2), (1000, 500,  200, 2)],
    'activation': ['relu'],
    'learning_rate_init': [0.001]
}
mlpc = mlpc_init()

grid_search = GridSearchCV(mlpc, param_grid, cv=3, scoring='accuracy', n_jobs=-1)

inicio = time.time()

grid_search.fit(X_train,y_train)

fin = time.time()

tiempo_ejecucion_MLPC = fin - inicio

print("Best parameters found:", grid_search.best_params_)
print("Best cross-validation accuracy:", grid_search.best_score_)
print(f"El tiempo de ejecucion fue de {tiempo_ejecucion_MLPC} segundos")

Entrenamiento MLP Classifier
Best parameters found: {'activation': 'relu', 'hidden_layer_sizes': (1000, 500, 2), 'learning_rate_init': 0.001}
Best cross-validation accuracy: 0.7396340724868442
El tiempo de ejecucion fue de 371.54574394226074 segundos


In [7]:
writeJson(tiempo_ejecucion_MLPC,grid_search.best_score_,"MLPC")

# Tiempos

In [8]:
print(f"Tiempo MLPC = {tiempo_ejecucion_MLPC:.2f} segundos")

Tiempo MLPC = 371.55 segundos
